In [1]:
import json
from pprint import pprint

In [2]:
file_path = '../data/document/역도/역도 훈련프로그램 구성 및 지도안_edited.json'
file_name = file_path.split('/')[-1]

with open(file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)
    data = data['edited']['elements']

In [3]:
from typing import TypedDict, Annotated, List, Dict, Tuple

class GraphState(TypedDict):
    filepath: Annotated[str, 'filepath']
    filepath_pdf: Annotated[str, 'pdf filepath']
    originData: Annotated[List[Dict], 'originData']
    documents: Annotated[List[Dict], 'documents']
    heading_structure: Annotated[List[str], 'heading structure. last heading is borderline']
    unused_elements: Annotated[List[Tuple[str, str]], 'unused elements']
    image_result: Annotated[List[Dict], 'result of image information extractor ']
    chart_result: Annotated[List[Dict], 'result of chart information extractor ']
    table_result: Annotated[List[Dict], 'result of table information extractor ']

In [4]:
state = GraphState(
    filepath = "../data/document/역도/역도 훈련프로그램 구성 및 지도안_edited.json",
    filepath_pdf = "../data/document/역도/역도 훈련프로그램 구성 및 지도안.pdf",
    originData = data,
    documents = data,
    heading_structure = ['heading1', 'heading2', 'heading3', 'heading4', 'heading5', 'default'],
    unused_elements = [('footer', 'default')]
)

In [5]:
def organize_relation(state: GraphState) -> Dict:
    '''
    relatedID가 [] 요소를 정렬하는 함수로, 두가지로 분류되며, 
    1. category가 caption인 경우
        - relatedID element에 {caption: 요소의 text} 형식으로 추가
        - 추가된 요소는 제거
    2. category가 caption이 아닌 경우
        - relatedID element의 바로 뒤에 요소를 복사
        - 복사된 요소는 제거
    '''           
            
    organized_data = []
    relations = []
    for element in state['documents']:
        if element['relatedID'] != []:
            relations.append(element)
        else:
            organized_data.append(element)
    
    for relation in relations:
        for related_id in relation['relatedID']:
            related_element_index = next((i for i, elem in enumerate(organized_data) if elem['id'] == related_id), None)
            if related_element_index is None:
                continue
            if relation['category'] == 'caption':
                organized_data[related_element_index]['caption'] = relation['text']
            else:
                organized_data.insert(related_element_index + 1, relation)
    return {'documents': organized_data}

In [6]:
def remove_unused_category(state: GraphState) -> Dict:
    '''
    state['unused_elements']에 있는 요소를 제거하는 함수.
    '''
    origin = state['documents'].copy()
    for element in origin:
        if (element['category'], element['class']) in state['unused_elements']:
            origin.remove(element)
    return {'documents': origin}


In [7]:
def document_extract(state: GraphState) -> Dict:
    """문서를 heading을 기준으로 chunking하는 함수
    
    Returns:
        Dict: {'documents': List[Dict]} 형태로 반환
        각 document는 다음 구조를 가짐:
        {
            'meta': {
                'heading': Dict,  # 현재 문서의 heading 상태
                'index': int     # chunk의 순서
            },
            'content': List[Dict]  # chunk에 포함된 elements
        }
    """
    elements = state['documents']
    current_heading = {heading: None for heading in state['heading_structure']}
    
    chunk_index = 0
    chunks = []
    contents = []

    for i, doc in enumerate(elements):
        if ('heading' in doc['category']):
            if contents != []:
                chunks.append({
                    'meta': {
                        'filepath': state['filepath'],
                        'heading': current_heading.copy(),
                        'index': chunk_index
                    },
                    'content': contents
                })
                contents = []
                chunk_index += 1
                
            current_heading[doc['class']] = doc['text']
            for heading in state['heading_structure'][state['heading_structure'].index(doc['class']) + 1:]:
                current_heading[heading] = None
        else:
            contents.append(doc)
    
    if contents != []:
        chunks.append({
            'meta': {
                'filepath': state['filepath'],
                'heading': current_heading.copy(),
                'index': chunk_index
            },
            'content': contents
        })
            
    return {'documents': chunks}

In [18]:
from langgraph.graph import StateGraph, END, START

workflow = StateGraph(GraphState)

workflow.add_node('organize_relation_node', organize_relation)
workflow.add_node('remove_unused_category_node', remove_unused_category)
workflow.add_node('document_extract_node', document_extract)

workflow.add_edge(START, 'organize_relation_node')
workflow.add_edge('organize_relation_node', 'remove_unused_category_node')
workflow.add_edge('remove_unused_category_node', 'document_extract_node')
workflow.add_edge('document_extract_node', END)

graph = workflow.compile()

In [19]:
from langchain_core.runnables import RunnableConfig

config = RunnableConfig(
    recursion_limit=5, 
    configurable={"thread_id": "Graph-Parser"}
)

# output = graph.stream(state, config=config)

state['documents'] = organize_relation(state)['documents']
state['documents'] = remove_unused_category(state)['documents']
state['documents'] = document_extract(state)['documents']

In [20]:
state["documents"][8]['meta']

{'filepath': '../data/document/역도/역도 훈련프로그램 구성 및 지도안_edited.json',
 'heading': {'heading1': 'Ⅴ.역도 훈련프로그램 구성 및 지도안',
  'heading2': '1. 훈련프로그램의 구성 원리',
  'heading3': '나. 훈련량의 설정 및 훈련프로그램의 실례',
  'heading4': '2) 운동강도(intensity：바의 평균중량)',
  'heading5': None,
  'default': None},
 'index': 8}

In [21]:
state["documents"][20]['content']

[{'category': 'paragraph',
  'coordinates': [{'x': 0.1242, 'y': 0.6147},
   {'x': 0.8781, 'y': 0.6147},
   {'x': 0.8781, 'y': 0.6948},
   {'x': 0.1242, 'y': 0.6948}],
  'id': 80,
  'page': 9,
  'relatedID': [],
  'class': 'default',
  'text': '초급자는 일주일에 3번 훈련하는 것이 바람직하며, 기록의 향상보다는 기술의 완\n성에 중점을 두어야 하며, 자발적으로 훈련에 참여 할 수 있도록 동기유발을 시키\n는 것이 매우 중요하다.'},
 {'category': 'table',
  'coordinates': [{'x': 0.128, 'y': 0.7299},
   {'x': 0.8757, 'y': 0.7299},
   {'x': 0.8757, 'y': 0.8806},
   {'x': 0.128, 'y': 0.8806}],
  'id': 82,
  'page': 9,
  'relatedID': [],
  'class': 'default',
  'text': '1일 세트×반복 2일 세트×반복 3일 세트×반복\n Warm-up  Warm-up  Warm-up \n Power Clean 5×5 Power Snatch 5×5 Clean 5×5\n Snatch 5×5 Jerk Balance 5×5 Press 5×5\n Balance 5×5 Clean Pull 5×5 Snatch Balance 5×5\n Snatch Pull 5×5 Front Squat 5×5 High Clean 5×5\n Back Squat 5×5   Back Squat 5×5',
  'caption': '표 62. 초급자의 1단계 훈련계획 예시'}]

In [22]:
#   import json
# state_json = json.dumps(state)

# with open('../data/document/역도/documentParseGraph_state.json', 'w', encoding='utf-8') as f:
#     f.write(state_json)